# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 13 2023 1:55PM,259197,10,MSP219430,"Methapharm, Inc.",Released
1,Mar 13 2023 1:55PM,259197,10,MSP219431,"Methapharm, Inc.",Released
2,Mar 13 2023 1:55PM,259197,10,MSP219434,"Methapharm, Inc.",Released
3,Mar 13 2023 1:55PM,259197,10,MSP219437,"Methapharm, Inc.",Released
4,Mar 13 2023 1:55PM,259197,10,MSP219464,"Methapharm, Inc.",Released
5,Mar 13 2023 1:55PM,259197,10,MSP219466,"Methapharm, Inc.",Released
6,Mar 13 2023 1:55PM,259197,10,MSP219467,"Methapharm, Inc.",Released
7,Mar 13 2023 1:55PM,259197,10,MSP219468,"Methapharm, Inc.",Released
8,Mar 13 2023 1:55PM,259197,10,MSP219469,"Methapharm, Inc.",Released
9,Mar 13 2023 1:55PM,259197,10,MSP219452,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,259187,Released,2
15,259189,Released,3
16,259196,Released,1
17,259197,Released,21
18,259198,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259187,NaN,NaN,2.0
259189,NaN,NaN,3.0
259196,NaN,NaN,1.0
259197,NaN,NaN,21.0
259198,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259138,8.0,7.0,0.0
259145,0.0,13.0,10.0
259150,0.0,11.0,3.0
259152,0.0,1.0,0.0
259159,0.0,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259138,8,7,0
259145,0,13,10
259150,0,11,3
259152,0,1,0
259159,0,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259138,8,7,0
1,259145,0,13,10
2,259150,0,11,3
3,259152,0,1,0
4,259159,0,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259138,8,7,
1,259145,,13,10
2,259150,,11,3
3,259152,,1,
4,259159,,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 13 2023 1:55PM,259197,10,"Methapharm, Inc."
21,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc."
22,Mar 13 2023 1:53PM,259196,10,"Nextsource Biotechnology, LLC"
23,Mar 13 2023 1:38PM,259189,10,Hush Hush
26,Mar 13 2023 1:36PM,259187,10,Emerginnova
28,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC"
52,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC"
61,Mar 13 2023 11:14AM,259168,18,Capsugel
62,Mar 13 2023 10:12AM,259159,10,"Methapharm, Inc."
64,Mar 13 2023 10:12AM,259159,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 13 2023 1:55PM,259197,10,"Methapharm, Inc.",,,21
1,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc.",,,1
2,Mar 13 2023 1:53PM,259196,10,"Nextsource Biotechnology, LLC",,,1
3,Mar 13 2023 1:38PM,259189,10,Hush Hush,,,3
4,Mar 13 2023 1:36PM,259187,10,Emerginnova,,,2
5,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",,19,5
6,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",,2,7
7,Mar 13 2023 11:14AM,259168,18,Capsugel,,1,
8,Mar 13 2023 10:12AM,259159,10,"Methapharm, Inc.",,2,1
9,Mar 13 2023 10:12AM,259159,10,Methapharm-G,,2,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 1:55PM,259197,10,"Methapharm, Inc.",21,,
1,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc.",1,,
2,Mar 13 2023 1:53PM,259196,10,"Nextsource Biotechnology, LLC",1,,
3,Mar 13 2023 1:38PM,259189,10,Hush Hush,3,,
4,Mar 13 2023 1:36PM,259187,10,Emerginnova,2,,
5,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",5,19,
6,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",7,2,
7,Mar 13 2023 11:14AM,259168,18,Capsugel,,1,
8,Mar 13 2023 10:12AM,259159,10,"Methapharm, Inc.",1,2,
9,Mar 13 2023 10:12AM,259159,10,Methapharm-G,1,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 1:55PM,259197,10,"Methapharm, Inc.",21,,
1,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc.",1,,
2,Mar 13 2023 1:53PM,259196,10,"Nextsource Biotechnology, LLC",1,,
3,Mar 13 2023 1:38PM,259189,10,Hush Hush,3,,
4,Mar 13 2023 1:36PM,259187,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 1:55PM,259197,10,"Methapharm, Inc.",21.0,NaN,NaN
1,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc.",1.0,NaN,NaN
2,Mar 13 2023 1:53PM,259196,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Mar 13 2023 1:38PM,259189,10,Hush Hush,3.0,NaN,NaN
4,Mar 13 2023 1:36PM,259187,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 1:55PM,259197,10,"Methapharm, Inc.",21.0,0.0,0.0
1,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc.",1.0,0.0,0.0
2,Mar 13 2023 1:53PM,259196,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Mar 13 2023 1:38PM,259189,10,Hush Hush,3.0,0.0,0.0
4,Mar 13 2023 1:36PM,259187,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2332556,49.0,30.0,0.0
15,259175,5.0,19.0,0.0
18,259168,0.0,1.0,0.0
19,518290,0.0,8.0,8.0
21,259198,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2332556,49.0,30.0,0.0
1,15,259175,5.0,19.0,0.0
2,18,259168,0.0,1.0,0.0
3,19,518290,0.0,8.0,8.0
4,21,259198,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,49.0,30.0,0.0
1,15,5.0,19.0,0.0
2,18,0.0,1.0,0.0
3,19,0.0,8.0,8.0
4,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,49.0
1,15,Released,5.0
2,18,Released,0.0
3,19,Released,0.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,21
Status,,,,,
Completed,0.0,0.0,0.0,8.0,0.0
Executing,30.0,19.0,1.0,8.0,0.0
Released,49.0,5.0,0.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,21
0,Completed,0.0,0.0,0.0,8.0,0.0
1,Executing,30.0,19.0,1.0,8.0,0.0
2,Released,49.0,5.0,0.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,21
0,Completed,0.0,0.0,0.0,8.0,0.0
1,Executing,30.0,19.0,1.0,8.0,0.0
2,Released,49.0,5.0,0.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()